In [6]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, html, datetime
import openpyxl
from openpyxl import Workbook 
from os import listdir
from os.path import isfile, join
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender 
from lib.script_generator import script_generator   
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')

In [7]:
def normalize(s): 
    s=re.sub('[^A-Za-z0-9\s]','',str(s))
    s=re.sub('\s{1,}','',str(s))
    return s.upper().strip()
normalize('Jump host access (e.g., the user accesses the HVA from an intermediate host) ')

'JUMPHOSTACCESSEGTHEUSERACCESSESTHEHVAFROMANINTERMEDIATEHOST'

In [8]:
pl=sql_todf("""
SELECT UPPER(LTRIM(RTRIM(DisplayValue))) [Selections], PK_PickList [ML_Value], PK_PickListType [PLT]
FROM vwPickLists  
WHERE PK_PickListType > 410
ORDER BY PK_PickListType DESC
""", connstr)  
s=list(pl.loc[pl['PLT']==491]['Selections'])
pl['NSelections'] = pl['Selections'].apply(normalize) 
pr = picklist_recommender(ctx.config['connstr'], reset_cache=True, picklist_where=' PK_PickListType > 410 ') 
#pr.normalize(' '.join(s))
pl.loc[pl['PLT']==491] 

423

In [ ]:
path=r'C:\Users\timko\Downloads\Questionnaire\\'
files = [f for f in listdir(path) if re.search('^HVA.*1\.xlsx', f)]
print('\n'.join(files))
xlfile=files[7]
metric='EP7a' 
xlfile  

In [ ]:
converts={
    'Selections':lambda x: re.sub('^(\s?\d{,2}\.)','',str(x)).upper().strip()
    , 'ML':lambda x: str(x).strip()
    , 'SML':lambda x: re.sub('[^A-Z0-9]','',str(x)).strip()  }
df=pd.read_excel(f'{path}{xlfile}', converters=converts,  sheet_name=4, header=2, usecols='B:I').reset_index()
df = df.rename(columns={'ML Score': 'ML_Score','ML Scored': 'ML_Scored' })
#df.drop(index=df.index[-15:], inplace=True)
for c in ['ID','Question','SML']:
    df[c] = df[c].replace(to_replace=[0, '0'], method='ffill')
if not 'ML' in df.columns:
    df['ML']=df["ML_Score"].apply(lambda x: str(x).strip())
    df['ML_Score']=df["ML_Scored"]
 
df["ID"].fillna(method='ffill', inplace=True) 
df.drop(columns=['Check Answer', 'index'], inplace=True)

df = df.loc[ df["ID"].str.contains('^\w\w\d(\w){0,5}$', na=False, regex=True) ]  
df = df.loc[ df["ML"].str.contains('\d', na=False,  regex=True, case=False) ]  
df['ML_Score'] = df['ML_Score'].apply(lambda s: re.sub('[^0-9\.]','',str(s)))
df['NSelections'] = df['Selections'].apply(normalize)
df.loc[df['ID']=='RA2b']['NSelections']
 

In [ ]:
s= list(df.loc[df['ID']=='RA2b']['NSelections'])
pr.recommend(s, normalizer=normalize)['PK_PicklistType'] 


In [ ]:
dfp=df.groupby(['ID'], as_index=False).agg({'NSelections':list})  
dfp['PLT']=dfp['Selections'].apply(lambda s: pr.recommend(s, normalizer=normalize)['PK_PicklistType'])
pr.to_cache() 

dff=pd.merge(df,dfp.loc[:,['ID','PLT']], how='left', left_on='ID', right_on='ID')
dff=pd.merge(dff,pl, how='left', left_on=['NSelections','PLT'], right_on=['NSelections','PLT'])
dff.ML_Value=dff.ML_Value.fillna(0).astype('int32').astype('str') 
dff

In [ ]:
df1=dff
df1.SML.fillna(method='ffill', inplace=True)
df1=df1.loc[df1['ML_Score'] != '0']
df1['sortorder'] = range(1, len(df1)+1)
df1=df1.loc[:, ['ID', 'SML', 'ML','ML_Score', 'ML_Scored', 'ML_Value', 'sortorder']]
sql=SQL_INSERT_FROM_DF(df1, TARGET='@SASCORING')
with open(f'{ctx.get_dest()}script.sql', 'w') as f:
    f.write('\n'.join(sql))
df1.loc[df1['ID']=='EP7a']

In [ ]:
def writefile():
    s=''
    template="2023-A-HVAENDPRO'\n\tBEGIN\n\n\tEND--'2023-A-HVAENDPRO'"
    with open(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\database\Sprocs\sp_HVASAScoreDash.sql', 'r', encoding='UTF-8') as f:
        s=f.read()
    m=re.search("(2023-A-HVAENDPRO.*--'2023-A-HVAENDPRO')",s, re.DOTALL)
    pholder=s[m.span(1)[0] :m.span(1)[1] ]
    s=s.replace(pholder, '')
    s
def check_pick(id='EP7a'):
    sel = list(df.loc[df['ID']==id]['Selections'])
    print(' '.join(sel))
    pr.recommend(sel, normalizer=normalize)